In [1]:
import grpc
import cv2
import numpy as np
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow_serving.apis import get_model_metadata_pb2
from tensorflow_serving.apis import predict_pb2
from tensorflow.contrib.util import make_tensor_proto

In [2]:
"""
Die Verbindungsdaten zum TensorFlow-Serving ModelServer
"""
host = "localhost"
port = "9000"
channel = grpc.insecure_channel("172.17.0.3:8500")
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
stub?

Type:           PredictionServiceStub
String form:    <tensorflow_serving.apis.prediction_service_pb2_grpc.PredictionServiceStub object at 0x7f16b4b93f98>
File:           /usr/local/lib/python3.6/site-packages/tensorflow_serving/apis/prediction_service_pb2_grpc.py
Docstring:     
open source marker; do not remove
PredictionService provides access to machine-learned models loaded by
model_servers.
Init docstring:
Constructor.

Args:
  channel: A grpc.Channel.


In [3]:
status = get_model_metadata_pb2.GetModelMetadataRequest()
status.model_spec.name = 'fruits'
status.model_spec.version.value = 1
status.metadata_field.append("signature_def")
response = stub.GetModelMetadata(status, 10)
response?

Type:            GetModelMetadataResponse
String form:    
model_spec {
           name: "fruits"
           version {
           value: 1
           }
           }
           metadata {
           key: "signature_def"
           <...> 377\377\377\377\377\377\377\377\377\001\022\002\010\020\032\032tensorflow/serving/predict"
           }
           }
           
File:            /usr/local/lib/python3.6/site-packages/tensorflow_serving/apis/get_model_metadata_pb2.py
Docstring:       <no docstring>
Class docstring: A ProtocolMessage


In [4]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'fruits'
request.model_spec.version.value = 1
img = cv2.imread("../1-deeplearning/test_images/Banana/Banana_wiki.jpg").astype(np.float32)
img = cv2.resize(img, (20, 20))
img = img / 255
x, y, c = img.shape
proto = make_tensor_proto(img, shape=[1, x, y, c])
request.inputs['input'].CopyFrom(proto)
result = stub.Predict(request)  # 10 secs timeout
result?

Type:            PredictResponse
String form:    
outputs {
           key: "prediction"
           value {
           dtype: DT_FLOAT
           tensor_shape {
           dim {
           <...> model_spec {
           name: "fruits"
           version {
           value: 1
           }
           signature_name: "serving_default"
           }
           
File:            /usr/local/lib/python3.6/site-packages/tensorflow_serving/apis/predict_pb2.py
Docstring:       <no docstring>
Class docstring: A ProtocolMessage


In [6]:
prediction = result.outputs['prediction'].float_val
max_value = max(prediction)
max_index = list(prediction).index(max_value)
print(max_value)

0.9750781655311584
